In [1]:

import torch
import torch.nn as nn
import torchvision.transforms as transforms
import timm
import torchsummary
import imageio
import numpy as np
import matplotlib.pyplot as plt
import os
from torch.cuda.amp import GradScaler, autocast

from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split, KFold

import sys
sys.path.append('../DataLoader')

from dataloader import SunImageDataset

from torch.func import stack_module_state
from torch.func import vmap

from lightning.fabric import Fabric

In [2]:
torch.set_float32_matmul_precision('medium')
fabric = Fabric(accelerator='cuda', devices=1, precision="bf16-mixed")
fabric.launch()
print(fabric.device)

Using bfloat16 Automatic Mixed Precision (AMP)


cuda:0


In [3]:
# Hyper-parameters
input_size = 224*224
hidden_size = 166
num_epochs = 20
batch_size = 2
learning_rate = 0.001
dropout = 0.5
# dropout = 0.6990787087509548

In [4]:
dataset = SunImageDataset(csv_file="D:\\dataset.csv", offset=0, transform=transforms.ToTensor())
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

# # Without Validation Set
# trainset, testset = torch.utils.data.Subset(dataset, range(train_size)), torch.utils.data.Subset(dataset, range(train_size, len(dataset)))
# trainloader = torch.utils.data.DataLoader(dataset=trainset, batch_size=batch_size, shuffle=True, num_workers=10)

# trainloader = fabric.setup_dataloaders(trainloader)

# With Validation Set
# Split dataset into training and test sets
train_indices, test_indices = train_test_split(range(len(dataset)), test_size=0.2, shuffle=False)

# Further split training set into training and validation sets
train_indices, val_indices = train_test_split(train_indices, test_size=0.25, shuffle=False)  # 0.25 x 0.8 = 0.2

trainset = torch.utils.data.Subset(dataset, train_indices)
valset = torch.utils.data.Subset(dataset, val_indices)
testset = torch.utils.data.Subset(dataset, test_indices)

trainloader = torch.utils.data.DataLoader(dataset=trainset, batch_size=batch_size, shuffle=True, num_workers=10)
valloader = torch.utils.data.DataLoader(dataset=valset, batch_size=batch_size, shuffle=False, num_workers=10)

trainloader = fabric.setup_dataloaders(trainloader)
valloader = fabric.setup_dataloaders(valloader)


# # Get a batch of training data
# dataiter = iter(trainloader)
# images, labels = next(dataiter)
# images = torch.stack(images)
# print(images.shape)
# print(labels.shape)

# print(images)
# print(labels)

In [5]:

class GmiSwinTransformer(nn.Module):
    def __init__(self, hidden_size: int):
        super(GmiSwinTransformer, self).__init__()
        
        # Batch normalization for 3 channels
        self.bn = nn.BatchNorm2d(3)
        
        # Initialize Swin Transformer
        self.pretrained_model = timm.create_model(
            'swin_base_patch4_window7_224',
            pretrained=True,
            num_classes=hidden_size
        )
        
        
        # Fully connected layers
        self.fc = nn.Sequential(
            nn.LeakyReLU(),
            nn.Linear(hidden_size*10, hidden_size),
            nn.Dropout(p=dropout),  # Added dropout probability
            nn.LeakyReLU(),
            nn.Linear(hidden_size, 1),
            nn.LeakyReLU()
        )
    
    def forward(self, images) -> torch.Tensor:
        """
        Batch should be in format:
        {
            'images': torch.FloatTensor((10, 1, 224, 224))
        }
        """
        
        batch_size = images.shape[0]
        
        images = images.reshape(-1, 1, 224, 224)
        images = torch.cat([images, images, images], dim=1)
        normalized_images = self.bn(images)
        features = self.pretrained_model(normalized_images)
        image_features = features.view(batch_size, -1)
        
        output = self.fc(image_features)
        return output

# Initialize model
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = GmiSwinTransformer(hidden_size=hidden_size).to(device)
model = GmiSwinTransformer(hidden_size=hidden_size)

# print(torchsummary.summary(model, (10, 1, 224, 224)))

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

model, optimizer = fabric.setup(model, optimizer)
model.train()

_FabricModule(
  (_forward_module): GmiSwinTransformer(
    (bn): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (pretrained_model): SwinTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
        (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      )
      (layers): Sequential(
        (0): SwinTransformerStage(
          (downsample): Identity()
          (blocks): Sequential(
            (0): SwinTransformerBlock(
              (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
              (attn): WindowAttention(
                (qkv): Linear(in_features=128, out_features=384, bias=True)
                (attn_drop): Dropout(p=0.0, inplace=False)
                (proj): Linear(in_features=128, out_features=128, bias=True)
                (proj_drop): Dropout(p=0.0, inplace=False)
                (softmax): Softmax(dim=-1)
              )
              (drop_pa

In [6]:
torch.cuda.empty_cache()

In [ ]:
# Training the model
n_total_steps = len(trainloader)
avg_train_loss_over_epochs = []
avg_val_loss_over_epochs = []

for epoch in range(num_epochs):
    train_losses = []
    val_losses = []
    
    # Training loop
    model.train()
    for i, (images, labels) in tqdm(enumerate(trainloader), desc="Training Progress", total=len(trainloader)):
        # Move images and labels to device
        images = torch.stack(images).float()
        images = images.permute(1, 0, 2, 3, 4)  # Change shape to [5, 10, 1, 224, 224]
        labels = labels.float()

        # Forward pass with autograd
        outputs = model(images).squeeze(1)
        loss = criterion(outputs, labels)
        tqdm.write(f"Epoch: {epoch+1}, Index: {i}, Loss: {loss.item():.4f}")

        # Backward pass and optimization
        optimizer.zero_grad()
        fabric.backward(loss)
        optimizer.step()
        # Store the loss
        train_losses.append(loss.item())
    
    # Store the average training loss for this epoch
    avg_train_loss_over_epochs.append(sum(train_losses) / len(train_losses))
    
    # Validation loop
    model.eval()
    with torch.no_grad():
        for images, labels in tqdm(valloader, desc="Validation Progress"):
            images = torch.stack(images).float()
            images = images.permute(1, 0, 2, 3, 4)
            labels = labels.float()
            outputs = model(images).squeeze(1)
            loss = criterion(outputs, labels)
            print("Validation Loss: ", loss.item())
            val_losses.append(loss.item())
    
    # Store the average validation loss for this epoch
    avg_val_loss_over_epochs.append(sum(val_losses) / len(val_losses))

# Plot loss over epochs
plt.figure()
plt.plot(range(1, num_epochs + 1), avg_train_loss_over_epochs, label='Average Training Loss', marker='o')
plt.plot(range(1, num_epochs + 1), avg_val_loss_over_epochs, label='Average Validation Loss', marker='o')
plt.xticks(range(1, num_epochs + 1))  # Ensure x-axis includes all epoch numbers
plt.xlabel('Epoch')
plt.ylabel('Average Loss')
plt.legend()
plt.title('Training and Validation Loss Over Epochs')
plt.show()

Training Progress:   0%|          | 0/307 [00:00<?, ?it/s]

Epoch: 1, Index: 0, Loss: 2.9206
Epoch: 1, Index: 1, Loss: 0.2089
Epoch: 1, Index: 2, Loss: 62.4478
Epoch: 1, Index: 3, Loss: 7.9942
Epoch: 1, Index: 4, Loss: 7.0289
Epoch: 1, Index: 5, Loss: 5.4620
Epoch: 1, Index: 6, Loss: 0.4657
Epoch: 1, Index: 7, Loss: 0.7289
Epoch: 1, Index: 8, Loss: 2.1132
Epoch: 1, Index: 9, Loss: 0.7197
Epoch: 1, Index: 10, Loss: 4.2164
Epoch: 1, Index: 11, Loss: 5.0883
Epoch: 1, Index: 12, Loss: 13.0442
Epoch: 1, Index: 13, Loss: 9.5281
Epoch: 1, Index: 14, Loss: 6.6037
Epoch: 1, Index: 15, Loss: 1.5012
Epoch: 1, Index: 16, Loss: 2.2566
Epoch: 1, Index: 17, Loss: 2.5071
Epoch: 1, Index: 18, Loss: 7.7257
Epoch: 1, Index: 19, Loss: 1.7571
Epoch: 1, Index: 20, Loss: 0.3626
Epoch: 1, Index: 21, Loss: 1.6751
Epoch: 1, Index: 22, Loss: 1.9915
Epoch: 1, Index: 23, Loss: 0.0313
Epoch: 1, Index: 24, Loss: 0.1284
Epoch: 1, Index: 25, Loss: 0.7214
Epoch: 1, Index: 26, Loss: 3.5963
Epoch: 1, Index: 27, Loss: 0.7338
Epoch: 1, Index: 28, Loss: 1.7465
Epoch: 1, Index: 29, L

Validation Progress:   0%|          | 0/103 [00:00<?, ?it/s]

Training Progress:   0%|          | 0/307 [00:00<?, ?it/s]

Epoch: 2, Index: 0, Loss: 2.3417
Epoch: 2, Index: 1, Loss: 1.6401
Epoch: 2, Index: 2, Loss: 2.9971
Epoch: 2, Index: 3, Loss: 0.2854
Epoch: 2, Index: 4, Loss: 2.7117
Epoch: 2, Index: 5, Loss: 0.2868
Epoch: 2, Index: 6, Loss: 0.4082
Epoch: 2, Index: 7, Loss: 4.3517
Epoch: 2, Index: 8, Loss: 1.2372
Epoch: 2, Index: 9, Loss: 0.1097
Epoch: 2, Index: 10, Loss: 1.5996
Epoch: 2, Index: 11, Loss: 1.7532
Epoch: 2, Index: 12, Loss: 2.9294
Epoch: 2, Index: 13, Loss: 1.5399
Epoch: 2, Index: 14, Loss: 3.8688
Epoch: 2, Index: 15, Loss: 2.6604
Epoch: 2, Index: 16, Loss: 0.1204
Epoch: 2, Index: 17, Loss: 0.2820
Epoch: 2, Index: 18, Loss: 1.5088
Epoch: 2, Index: 19, Loss: 3.9357
Epoch: 2, Index: 20, Loss: 1.5324
Epoch: 2, Index: 21, Loss: 1.8409
Epoch: 2, Index: 22, Loss: 3.8735
Epoch: 2, Index: 23, Loss: 2.0441
Epoch: 2, Index: 24, Loss: 3.2741
Epoch: 2, Index: 25, Loss: 2.0183
Epoch: 2, Index: 26, Loss: 5.3922
Epoch: 2, Index: 27, Loss: 4.5136
Epoch: 2, Index: 28, Loss: 0.4267
Epoch: 2, Index: 29, Los

...

Training Progress:   0%|          | 0/307 [00:00<?, ?it/s]

Epoch: 10, Index: 0, Loss: 0.2700
Epoch: 10, Index: 1, Loss: 3.7155
Epoch: 10, Index: 2, Loss: 0.3566
Epoch: 10, Index: 3, Loss: 7.6319
Epoch: 10, Index: 4, Loss: 2.6371
Epoch: 10, Index: 5, Loss: 16.5236
Epoch: 10, Index: 6, Loss: 3.2116
Epoch: 10, Index: 7, Loss: 0.2724
Epoch: 10, Index: 8, Loss: 0.2444
Epoch: 10, Index: 9, Loss: 0.2560
Epoch: 10, Index: 10, Loss: 2.0009
Epoch: 10, Index: 11, Loss: 0.2403
Epoch: 10, Index: 12, Loss: 3.2734
Epoch: 10, Index: 13, Loss: 1.7839
Epoch: 10, Index: 14, Loss: 0.1503
Epoch: 10, Index: 15, Loss: 3.1935
Epoch: 10, Index: 16, Loss: 2.7153
Epoch: 10, Index: 17, Loss: 3.8039
Epoch: 10, Index: 18, Loss: 0.3594
Epoch: 10, Index: 19, Loss: 1.2317
Epoch: 10, Index: 20, Loss: 1.3043
Epoch: 10, Index: 21, Loss: 2.9155
Epoch: 10, Index: 22, Loss: 4.2536
Epoch: 10, Index: 23, Loss: 0.4853
Epoch: 10, Index: 24, Loss: 1.1201
Epoch: 10, Index: 25, Loss: 0.5473
Epoch: 10, Index: 26, Loss: 0.8124
Epoch: 10, Index: 27, Loss: 0.7939
Epoch: 10, Index: 28, Loss: 2

Validation Progress:   0%|          | 0/103 [00:00<?, ?it/s]

Training Progress:   0%|          | 0/307 [00:00<?, ?it/s]

Epoch: 11, Index: 0, Loss: 1.6976
Epoch: 11, Index: 1, Loss: 0.3148
Epoch: 11, Index: 2, Loss: 1.9809
Epoch: 11, Index: 3, Loss: 1.2043
Epoch: 11, Index: 4, Loss: 3.5435
Epoch: 11, Index: 5, Loss: 8.4871
Epoch: 11, Index: 6, Loss: 0.3380
Epoch: 11, Index: 7, Loss: 1.0701
Epoch: 11, Index: 8, Loss: 2.8066
Epoch: 11, Index: 9, Loss: 1.8173
Epoch: 11, Index: 10, Loss: 0.7958
Epoch: 11, Index: 11, Loss: 0.3423
Epoch: 11, Index: 12, Loss: 0.9439
Epoch: 11, Index: 13, Loss: 0.0762
Epoch: 11, Index: 14, Loss: 0.4518
Epoch: 11, Index: 15, Loss: 1.6553
Epoch: 11, Index: 16, Loss: 1.3376
Epoch: 11, Index: 17, Loss: 1.7786
Epoch: 11, Index: 18, Loss: 0.7237
Epoch: 11, Index: 19, Loss: 0.2008
Epoch: 11, Index: 20, Loss: 0.3137
Epoch: 11, Index: 21, Loss: 0.2318
Epoch: 11, Index: 22, Loss: 1.2239
Epoch: 11, Index: 23, Loss: 0.4982
Epoch: 11, Index: 24, Loss: 3.3868
Epoch: 11, Index: 25, Loss: 1.5991
Epoch: 11, Index: 26, Loss: 0.4283
Epoch: 11, Index: 27, Loss: 0.7744
Epoch: 11, Index: 28, Loss: 4.

In [ ]:
# Test the model
model.eval()
testloader = torch.utils.data.DataLoader(dataset=testset, batch_size=batch_size, shuffle=False, num_workers=10)
testloader = fabric.setup_dataloaders(testloader)
test_losses = []

with torch.no_grad():
    for images, labels in tqdm(testloader, desc="Testing Progress"):
        images = torch.stack(images).float()
        images = images.permute(1, 0, 2, 3, 4)
        labels = labels.float()
        outputs = model(images).squeeze(1)
        loss = criterion(outputs, labels)
        test_losses.append(loss.item())
        print("Test loss", test_losses)

avg_test_loss = sum(test_losses) / len(test_losses)
print(f'Average test loss: {avg_test_loss:.4f}')

Testing Progress:   0%|          | 0/103 [00:00<?, ?it/s]